# importing the dataset

we downloaded the dataset from kaggle

https://www.kaggle.com/uciml/sms-spam-collection-dataset?select=spam.csv

we will use this dataset to classify whether an sms is a spam or ham

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
plt.style.use("seaborn-pastel")
sns.set_theme(palette="dark",style="dark" )

In [ ]:
#change the encoding to utf-8 using ms excel first
#or use  ---> encoding='latin1' <--- as an argument in the below line of code
df = pd.read_csv("../input/sms-spam-collection-dataset/spam.csv", index_col=0, usecols=[0,1])

In [ ]:
df

In [ ]:
df.index.name = "class"

In [ ]:
df.columns = ["Text"]

In [ ]:
df.columns

In [ ]:
df.index.value_counts()

#we need to stratifically divide our train test data.

In [ ]:
4825/(4825+747)
#86.5% ham
#13.5% spam

In [ ]:
sns.barplot(x=df.index.unique(), y=df.index.value_counts())

# seperating the feature and target column

In [ ]:
# this is the first step we need to perform before splitting the dataset

x, y = df.iloc[:,0], df.index

In [ ]:
x = pd.DataFrame(x)

In [ ]:
y = pd.DataFrame(y)

In [ ]:
y.head(2)

In [ ]:
x.index = y.index

In [ ]:
x.head(2)

# train test splitting

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y )

In [ ]:
y_train.value_counts()

In [ ]:
3859/4457

In [ ]:
y_test.value_counts()

In [ ]:
966/(966+149)

we stratified our splitting using the y df(target column) that we had. we cross checked the ratio, and it's exactly what we needed

In [ ]:
x_train

# vectorisation 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

x_train_series = x_train.squeeze()
lst = x_train_series.tolist()
cv = CountVectorizer(input = lst,
        lowercase=True,
        stop_words='english' #for removing stop words
                     )

In [ ]:
x_train_prepared = cv.fit_transform(lst)

In [ ]:
x_train_prepared.shape

In [ ]:
lst_test = (x_test.squeeze()).tolist()
x_test_prepared = cv.transform(lst_test)

In [ ]:
x_test_prepared

In [ ]:
y_train = (np.array(y_train)).ravel()

# training the model

In [ ]:
from sklearn.svm import SVC
svm_clf = SVC()

svm_clf.fit(x_train_prepared, y_train)

# performance measure

In [ ]:
some_data = x_train_prepared[47:53]
some_labels = y_train[47:53]

In [ ]:
print("predictions:", svm_clf.predict(some_data))
print("labels:", some_labels)

In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_predict = cross_val_predict(svm_clf, x_train_prepared, np.array(y_train).ravel(), cv=3)

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, classification_report, accuracy_score
cm = confusion_matrix(y_train, y_train_predict)
print(cm)

In [ ]:
f1_score(y_train, y_train_predict, average='weighted')

In [ ]:
print(classification_report(y_train, y_train_predict))

In [ ]:
accuracy_score(y_train, y_train_predict)

In [ ]:

#heatmap for confusion matrix
group_names = ['True Neg','False Pos','False Neg','True Pos']

labels = np.asarray(group_names).reshape(2,2)
sns.heatmap(cm, annot=labels, fmt='', cmap='PuRd')

# accuracy on test dataset

In [ ]:
y_test_predict = svm_clf.predict(x_test_prepared)

In [ ]:
accuracy_score(y_test, y_test_predict)

In [ ]:
print("predictions:", y_test_predict[:5])
print("actual labels:", y_test[:5])

In [ ]:
y_test_predict = cross_val_predict(svm_clf, x_test_prepared, np.array(y_test).ravel())
cm_test = confusion_matrix(y_test, y_test_predict)
print(cm_test)

In [ ]:
f1_score(y_test, y_test_predict, average='weighted')

In [ ]:
print(classification_report(y_test, y_test_predict))

In [ ]:

#heatmap for confusion matrix
group_names = ['True Neg','False Pos','False Neg','True Pos']

labels = np.asarray(group_names).reshape(2,2)
sns.heatmap(cm_test, annot=labels, fmt='', cmap='PuRd')